In [29]:
from collections import Counter
import numpy as np
import re

# *1. Jaccad 유사도*  
![Jaccad 유사도](./images/jaccad_sim.png)  

# *2. Cosine 유사도*
![Cosine 유사도](./images/jaccad_sim.png)  

In [150]:
## 단어 전처리해주는 함수 | 정규 표현식으로 알파벳과 숫자만 남기도록 함
word_preprocessing = lambda word: re.sub('[^a-zA-Z0-9]', '', word)

class similarity:

    def __init__(self, word1, word2):
        self.word1 = word1
        self.word2 = word2

    ## 단어 전처리해주는 함수 | 정규 표현식으로 알파벳과 숫자만 남기도록 함
    def preprocessing(self, word): 
        return re.sub('[^a-zA-Z0-9]', '', word)

    ## 단어를 벡터화 시키는 함수
    def word_vectorizer(self, word):
        word = self.preprocessing(word)
        word_count = Counter(word.lower())

        ## ascii | 0 ~ 9, a ~ z 까지의 문자열을 리스트에 저장 (ascii 코드표에 대응시킴)
        ascii = [chr(idx) for idx in range(48, 58)] + [chr(idx) for idx in range(97, 127)]
        alphabets = {alpha: 0 for alpha in ascii}

        ## 단어에 있는 알파벳 갯수만을 반환시켜줌.
        for k, v in word_count.items(): alphabets[k] = v
        return np.array([num for num in alphabets.values()])

    ## 텍스트를 벡터화 시키는 함수
    def text_vectorizer(self):
        
        ## 공백을 기준으로 텍스트를 나눔.
        word1, word2 = self.word1.split(' '), self.word2.split(' ')
        
        ## 정규표현식을 통해 영어, 숫자만 남기도록 전처리.
        word1 = [self.preprocessing(word) for word in word1]
        word2 = [self.preprocessing(word) for word in word2]
        
        ## 정규표현식을 통해 삭제되어 생성된 공백문자 제거
        ## e.g)                   words = ['abcd', '무야호!', 'defg']
        ##      (정규표현식 거침) words = ['abcd', '', 'defg']
        ##            (공백 제거) words = ['abcd', 'defg']
        
        words1 = [word for word in word1 if word not in '']
        words2 = [word for word in word2 if word not in '']

        ## 첫 번째 텍스트와 두 번째 텍스트에 있는 단어들로 딕셔너리 생성
        words1_dict, words2_dict = {word: 0 for word in words1}, {word: 0 for word in words2}
        
        ## {텍스트 : 개수}로 구성된 딕셔너리
        words1, words2 = Counter(words1), Counter(words2)
        
        ## words1_dict와 words2_dict를 합쳐 하나의 딕셔너리로 생성
        vector1, vector2 = {**words1_dict, **words2_dict},  {**words1_dict, **words2_dict}
        
        ## vector 딕셔너리에 텍스트에 있는 단어 개수 입력하는 부분
        for word, num in words1.items(): vector1[word] = num
        for word, num in words2.items(): vector2[word] = num
        
        ## 텍스트 벡터화 시켜주는 부분
        vector1 = np.array([num for num in vector1.values()])
        vector2 = np.array([num for num in vector2.values()])

        return vector1, vector2

    ## Jaccad 유사도 구하는 함수
    def jaccad_sim(self):

        word1 = self.preprocessing(self.word1)
        word2 = self.preprocessing(self.word2)

        intersection = set(word1).intersection(set(word2))
        union        = set(word1).union(set(word2))

        return len(intersection) / len(union)


    ## 코사인 유사도 구하는 함수
    def cosine_sim(self, task = 'word'):

        if 'word' in task.lower():
          ## 입력받은 단어 두 개를 벡터화 시켜줌.
          vector1 = self.word_vectorizer(self.word1)
          vector2 = self.word_vectorizer(self.word2)

        elif 'text' in task.lower():
            vector1, vector2 = self.text_vectorizer()

        ## 코사인 유사도 구해주는 부분
        try: cosine = np.dot(vector1, vector2) / (np.linalg.norm(vector1)*np.linalg.norm(vector2))
        except: cosine = 0.0

        return cosine

In [146]:
sim = similarity('calcif', 'calcification')
sim2 = similarity('calclifcation', 'calcification')

In [147]:
jaccad1 = sim.jaccad_sim()
jaccad2 = sim2.jaccad_sim()

cosine1 = sim.cosine_sim()
cosine2 = sim2.cosine_sim()

print(f'jaccad1 : {jaccad1}, cosine1 : {cosine1}')
print(f'jaccad2 : {jaccad2}, cosine2 : {cosine2}')

jaccad1 : 0.625, cosine1 : 0.884537962671703
jaccad2 : 1.0, cosine2 : 0.9622504486493763


In [153]:
sim = similarity('micro calcification', 'macro calcification')

print(sim.text_vectorizer())
print(sim.cosine_sim(task='text'))

(array([1, 1, 0]), array([0, 1, 1]))
0.4999999999999999


In [158]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(['micro calcification', 'macro calcification'])

cosine_similarity(tfidf_matrix)

array([[1.        , 0.33609693],
       [0.33609693, 1.        ]])